## IMDB data extraction

data set:
https://datasets.imdbws.com/

In [13]:
import pandas as pd

In [14]:
movies = pd.read_csv("MovieSummaries/movie.metadata.tsv", delimiter='\t',header=None)
movies.columns = ['Wikipedia movie ID','Freebase movie ID','Movie name','Movie release date','Movie box office revenue','Movie runtime','Movie languages','Movie countries','Movie genres']
#Removes all movies that have no box office informations
movies = movies.dropna(subset=['Movie box office revenue'])

In [15]:
# imdb1 has ratings, imdb identifier and number of votes
imdb1 = pd.read_csv('title.ratings.tsv', delimiter='\t')
# imdb2 has genres, title, year, etc
imdb2 = pd.read_csv('title.basics.tsv', delimiter='\t', low_memory=False)


In [16]:
imdb_df = pd.merge(imdb1, imdb2, on='tconst', how='inner')

In [17]:
# in the imdb data set they includes series and a lot of other things.
#  This removes a lot of movies that are not in the MCU dataset
imdb_df = imdb_df[(imdb_df.titleType == 'movie')|(imdb_df.titleType == 'tvMovie')]
# Removing all the movies that have less than 30 votes
imdb_df= imdb_df[imdb_df['numVotes'] >= 30]
# making the titles of the movies in the same format for both datasets
# for example in one data set a movie can be called "The matrix" and in the other "The Matrix"
imdb_df.loc[:, 'primaryTitle'] = imdb_df['primaryTitle'].str.capitalize()
movies["Movie name"] = movies["Movie name"].str.capitalize()
movies["Movie name"] = movies["Movie name"].str.replace(r'[éè]', 'e', regex=True)

In [18]:
# removing all the columns that are not needed
imdb_df = imdb_df.drop(columns=['numVotes' ,'titleType', 'isAdult', 'endYear', 'originalTitle', 'genres', 'tconst'])

In [ ]:
# merging the data sets on our movie data set.
#  This will create duplicates since multiple movies can have the same name.
merged_df = pd.merge(imdb_df, movies, left_on='primaryTitle', right_on='Movie name', how='right')

In [ ]:
# Yreating a column that has the year of the movie release so 
# that we can remove duplicates which have the same name but different release year
merged_df["Movie release year"] = merged_df["Movie release date"].str[:4]

In [52]:
# trying to remove duplicates
merged_df = merged_df[merged_df['Movie release year']==merged_df['startYear']]
merged_df.index = merged_df['Wikipedia movie ID']

In [ ]:
#Removing additional duplicated based on run time differences
merged_df["runtimeMinutes"] = pd.to_numeric(merged_df["runtimeMinutes"], errors='coerce', downcast='integer') 
problem_df1 = merged_df[merged_df["Wikipedia movie ID"].duplicated(keep=False)]
problem_df2 = problem_df1[problem_df1["Movie name"].duplicated(keep=False)]
problem_df3 = problem_df2[problem_df2["Wikipedia movie ID"].duplicated(keep=False)]
problem_df4 = problem_df3[abs(problem_df3["runtimeMinutes"]-problem_df3["Movie runtime"])>20]
merged_df[~merged_df.index.isin(problem_df4.index)]

# drop colllumn that are not needed
merged_df = merged_df.drop(columns=['primaryTitle', 'Movie release year', 'startYear', 'Movie release date', 'runtimeMinutes'])


# The Cleaned data

In [ ]:
merged_df